# **Covid-19 Vaccines - Sentiment Analysis & Time Series**
Notebook for the second project for the Machine Learning Complements course (CAC).

## **Introduction**


## Imports

The following libraries will be used in this project:

In [ ]:
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import utils as ut
import warnings
import seaborn as sns
import contractions
import nltk
import plotly.express as px
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.classify import NaiveBayesClassifier
nltk.download('vader_lexicon')
nltk.download('movie_reviews')
warnings.simplefilter(action='ignore')


## Load Data

In [ ]:
df_tweets = pd.read_csv('tweets.csv')

## Initial Observations

The dataset contains a single file: `tweets.csv`.

In this section we will take a look at the first few rows of each file to get a better understanding of the data, and do some initial data exploration.

In [ ]:
ut.initial_obs(df_tweets)

## Data Understanding

## Data Pre-Processing
We can see that many attributes are not really relevant for the kind of work we will be doing. Therefore, we'll just selec tthe most relevant ones.

In [ ]:
df_tweets = df_tweets[['id','user_location','date','text','hashtags','user_followers']]
pd.set_option('display.max_colwidth', None)

df_tweets['text'].head(5)
df_tweets['orig_text'] = df_tweets['text']

### Removing Spaces within the text
When removing spaces within the text, ensure seamless integration of words for enhanced readability and processing efficiency.

In [ ]:
df_tweets['text'] = df_tweets['text'].apply(ut.trim_text)
df_tweets['text'].head(5)

### Contractions Mapping
Contractions mapping simplifies language by expanding contractions like "can't" to "cannot" for consistent analysis and interpretation.

In [ ]:
df_tweets['text'] = df_tweets['text'].apply(contractions.fix)
df_tweets['text'].head(5)

### Cleaning HTML
Cleaning HTML tags from text data streamlines content for NLP tasks, preventing interference from markup elements.

In [ ]:
df_tweets['text'] = df_tweets['text'].apply(lambda x:re.sub(r"http\S+", "", x))
df_tweets['text'].head(5)

### Emojis & Emotion Handling
Emojis and emotion handling enrich text analysis by capturing nuances of sentiment and expression for deeper understanding. We thought about removing them initially, however their presence may be crucial to identify sentiments within the text.

In [ ]:
pattern = re.compile(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251\U0001F004\U0001F0CF\U0001F170-\U0001F251\U0001F600-\U0001F64F\U0001F680-\U0001F6FF]+', flags=re.UNICODE)

# Find examples in df_tweets['text'] that have emojis
emojis_examples = df_tweets[df_tweets['text'].str.contains(pattern, na=False)][0:5]

for index in emojis_examples.index:
    print(df_tweets.loc[index, 'text'])

df_tweets['text'] = df_tweets['text'].apply(ut.convert_emojis_to_text)

print('\n')

for index in emojis_examples.index:
    emoji_text = df_tweets.loc[index, 'text']
    print(emoji_text)

### Handling Twitter Handles (@) & Hashtags
Handling Twitter handles (@) and hashtags facilitates contextual analysis and topic extraction in social media text. We removed the twitter handle, as they mostly are used to identify persons therefore they are not very important in this matter. On the other hand, hashtags may indicate sentiments or other important informations like topics. e.g #sad, #happy or #astrozeneca

In [ ]:
df_tweets['text'] = df_tweets['text'].apply(ut.remove_twitter_handles_hashtags)

for index in emojis_examples.index:
    emoji_text = df_tweets.loc[index, 'text']
    print(emoji_text)

### Convert text to lower-case
Converting all the characters to lower case so that words in different forms can be interpreted as the same. The problem with this is that in social media people may use upper-case to express sentiments, e.g SAD, HAPPY.

Here we also remove special characters, keeping only characters.

In [ ]:
df_tweets['text'] = df_tweets['text'].apply(ut.remove_special_characters)

df_tweets['text'] = df_tweets['text'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
df_tweets['text'] = df_tweets['text'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

df_tweets['text'] = df_tweets['text'].str.lower()

### Tokenization
Tokenization breaks down text into individual units, such as words or phrases, enabling granular analysis and feature extraction. We also remove stopwords, meaning words that often appear within the text and don't add any meaning to it.

In [ ]:
df_tweets['tokenized_text'] = df_tweets['text'].apply(lambda x: word_tokenize(x))
df_tweets['tokenized_text'] = df_tweets['tokenized_text'].apply(ut.remove_stopwords)
df_tweets['token_text'] = df_tweets['tokenized_text'].apply(lambda text: " ".join(text))


df_tweets['tokenized_text'].head(5)

### Stemming
Stemming typically chops off prefixes and/or suffixes of words to derive the root form. It's a simpler and faster process compared to lemmatization. However, stemming doesn't always result in valid words. For instance, "running" might be stemmed to "runn," which isn't a valid word.

In [ ]:
stemmer = PorterStemmer()
df_tweets['stemmed_text'] = df_tweets['tokenized_text'].apply(lambda x: [stemmer.stem(word) for word in x])

df_tweets['stemmed_text'].head(5)

### Lemmatization
Lemmatization, on the other hand, involves resolving words to their dictionary form, known as the lemma. It uses lexical knowledge bases to ensure that the root form returned is a valid word. For example, "am," "are," and "is" would all be lemmatized to "be." Lemmatization is generally more accurate than stemming but can be slower due to its linguistic complexity.

In [ ]:
lemmatizer = WordNetLemmatizer()
df_tweets['lemmatized_text'] = df_tweets['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

df_tweets['lemmatized_text'].head(5)
df_tweets['clean_text'] = df_tweets['lemmatized_text'].apply(lambda text: " ".join(text))


## Sentiment Analysis - Using VADER & TextBlob

In [ ]:
sid = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    vader_scores = sid.polarity_scores(text)['compound']
    if vader_scores >= 0.05:
        sentiment = 'Positive'
    elif vader_scores <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
    return sentiment, vader_scores

df_tweets['sentiment'], df_tweets['vader_score'] = zip(*df_tweets['text'].apply(analyze_sentiment))
#df_tweets['sentiment'] = df_tweets['sentiment'].replace({'Positive': 1, 'Neutral': 0, 'Negative': -1})

In [ ]:
ut.plot_sentiments(df_tweets)

### WordClouds

#### Positive Sentiment - WordCloud

In [ ]:
positive_tweets = df_tweets[df_tweets['sentiment'] == "Positive"]
negative_tweets = df_tweets[df_tweets['sentiment'] == "Negative"]
neutral_tweets = df_tweets[df_tweets['sentiment'] == "Neutral"]

ut.generate_word_cloud(positive_tweets['token_text'], 'Positive Sentiment Word Cloud')
positive_tweets['clean_text'].head(5)

In [ ]:
ut.common_words(positive_tweets, 50)

#### Neutral Sentiment - WordCloud

In [ ]:
ut.generate_word_cloud(neutral_tweets['token_text'], 'Neutral Sentiment Word Cloud')
neutral_tweets['clean_text'].head(5)

In [ ]:
ut.common_words(neutral_tweets, 50)

#### Negative Sentiment - WordCloud

In [ ]:
ut.generate_word_cloud(negative_tweets['token_text'], 'Negative Sentiment Word Cloud')
negative_tweets['clean_text'].head(5)

In [ ]:
ut.common_words(negative_tweets, 50)

### N-Gram Analysis by sentiment

#### Uni-Gram

In [ ]:
ut.plot_n_grams(df_tweets, 1)

#### Bi-Gram

In [ ]:
ut.plot_n_grams(df_tweets, 2)

#### Tri-Gram

In [ ]:
ut.plot_n_grams(df_tweets, 3)

### Plotting Average Word Amount by Sentiment

In [ ]:
ut.plot_avg_word_length_distribution_multi(positive_tweets, neutral_tweets, negative_tweets)



## Sentiment Analysis - Typical ML Approach
As we can see our dataset is not labelled, therefore we can't separate it into train/test and just train a model. What we will do is train a model using a labelled tweet dataset (the theme of both dataset would be similar, so we can use it for training) and then test on our dataset.

In [ ]:
"""
training_df = pd.read_csv('train_tweets.csv')
training_df = training_df[training_df['new_sentiment'].notna()]
training_df = training_df[['old_text','new_sentiment']]


training_df = ut.pre_process_pipeline(training_df,'old_text')
training_df.rename(columns={'new_sentiment': 'sentiment'}, inplace=True)
training_df['sentiment'] = training_df['sentiment'].replace({'positive': 1, 'neutral': 0, 'negative': -1})

ut.initial_obs(training_df)"""

### Generating Training-Features

In [ ]:
"""
train_features, test_features = ut.generate_features(training_df,df_tweets,'tfidf')

#TF-IDF Results:
pred_nb_tfidf, pred_svm_tfidf = ut.predict_labels(train_features,training_df['sentiment'].values,test_features)

df_tweets['predicted_tfidf_NB'] = pred_nb_tfidf
df_tweets['predicted_tfidf_SVM'] = pred_svm_tfidf


df_eval = df_tweets.copy()
df_eval = df_eval[['clean_text','text','sentiment','predicted_tfidf_NB','predicted_tfidf_SVM']]

import seaborn as sns

sentiment_counts = df_eval['predicted_tfidf_NB'].value_counts()
plt.figure(figsize=(8, 6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette="viridis")
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()"""

## Topic Modelling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

"""
vectorizer = CountVectorizer(max_features=1000, 
                             stop_words='english')

doc_term_matrix = vectorizer.fit_transform(df_tweets['text'])"""

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_tweets['clean_text'])

# Step 3: Apply LDA
num_topics = 8  # You can adjust the number of topics
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(tfidf_matrix)

# Step 4: Display the topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx+1}:")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 15  # Number of top words to display for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
display_topics(lda, feature_names, no_top_words)

## Geo-Spatial Sentiment Analysis

## Time-Series Analysis

In [ ]:
# Function to filter the data to a single date and print tweets from users with the most followers
def date_filter(df, date):
    return df[df['date'].astype(str)==date].sort_values('user_followers', ascending=False)[['date' ,'orig_text']]

def date_printer(df, dates, num=10): 
    for date in dates:
        display(date_filter(df, date).head(num))
        
# Get tweets for vaccine Pfizer,Pfizer; Sinopharm;Sinovac;Moderna;AstraZeneca;Covaxin;Sputnik V.
df_tweets_pz = df_tweets[df_tweets['clean_text'].str.contains('pfizer', case=False, na=False)]
df_tweets_sinopharm = df_tweets[df_tweets['clean_text'].str.contains('sinopharm', case=False, na=False)]
df_tweets_sinovac = df_tweets[df_tweets['clean_text'].str.contains('sinovac', case=False, na=False)]
df_tweets_moderna = df_tweets[df_tweets['clean_text'].str.contains('moderna', case=False, na=False)]
df_tweets_astrazeneca = df_tweets[df_tweets['clean_text'].str.contains('astrazeneca', case=False, na=False)]
df_tweets_covaxin = df_tweets[df_tweets['clean_text'].str.contains('covaxin', case=False, na=False)]
df_tweets_sputnik = df_tweets[df_tweets['clean_text'].str.contains('sputnik', case=False, na=False)]

print('Number of tweets for Pfizer:', len(df_tweets_pz))
print('Number of tweets for Sinopharm:', len(df_tweets_sinopharm))
print('Number of tweets for Sinovac:', len(df_tweets_sinovac))
print('Number of tweets for Moderna:', len(df_tweets_moderna))
print('Number of tweets for AstraZeneca:', len(df_tweets_astrazeneca))
print('Number of tweets for Covaxin:', len(df_tweets_covaxin))
print('Number of tweets for Sputnik V:', len(df_tweets_sputnik))

### Sentiment Over Time

In [ ]:
df_tweets['date'] = pd.to_datetime(df_tweets['date'])
df_tweets['date'] = df_tweets['date'].dt.date

# Get counts of number of tweets by sentiment for each date
timeline = df_tweets.groupby(['date', 'sentiment']).agg(**{'tweets': ('id', 'count')}).reset_index().dropna()

# Plot results

fig = px.line(timeline, x='date', y='tweets', color='sentiment', category_orders={'sentiment': ['neutral', 'negative', 'positive']},color_discrete_sequence=[ '#EF553B','#636EFA', '#00CC96'], title='Number of Tweets by Sentiment Over Time')
fig.show()

There are some spikes in the data, which may be due to some events that happened in the world. Let's investigate them further.

In [ ]:
spike = df_tweets[df_tweets['date'].astype(str)=='2021-03-01']
spike['user_location'].value_counts().head(10)

In [ ]:
spike = spike.sort_values('user_location', ascending=False)
spike['orig_text'].head(10)

It looks like the Prime Minister of India took the first dose of the Covid 19 vaccine on March 1st, 2021. This event caused a spike in the number of tweets and we can see that the sentiment is mostly positive.

#### Covaxin

In [ ]:
def filtered_timeline(df, vax, title):
    df = df.dropna()
    title_str = 'Timeline showing sentiment of tweets about the '+title+' vaccine'
    vac_tweets = df_tweets[df_tweets['clean_text'].str.contains(vax, case=False, na=False)]
    
    timeline = vac_tweets.groupby(['date', 'sentiment']).agg(**{'tweets': ('id', 'count')}).reset_index()
    fig = px.line(timeline, x='date', y='tweets', color='sentiment', color_discrete_map={'Positive': '#00CC96', 'Negative': '#EF553B', 'Neutral': '#636EFA'},title=title_str)

    fig.show()
    return vac_tweets

covaxin = filtered_timeline(df_tweets, 'covaxin', title='Covaxin')

In [ ]:
covaxin_spike = covaxin[covaxin['date'].astype(str)=='2021-11-03']
print('Number of tweets on 2021-11-03:', len(covaxin_spike))
covaxin_spike['user_location'].value_counts().head(10)

In [ ]:
covaxin_spike = covaxin[covaxin['user_location']=='India']

date_printer(covaxin_spike, ['2021-11-03'], num=5)

After a brief investigation we found out that Covaxin was approved for emergency use by WHO (World Health Organization) on May 3rd, 2021. This led to a positive spike in the tweets.

#### Sinopharm

In [ ]:
sinopharm = filtered_timeline(df_tweets, 'sinopharm', title='Sinopharm')


In [ ]:
sinopharm_spike = sinopharm[sinopharm['date'].astype(str)=='2021-08-13']
print('Number of tweets on July 13th:', len(sinopharm_spike))


print(sinopharm_spike['orig_text'].head(10))

# Count how many duplicate tweets there are
print('Number of duplicate tweets:', len(sinopharm_spike[sinopharm_spike.duplicated(subset='clean_text')]))

For the Sinopharm vaccine there was one major event that happened. An 89 year old man died after taking the vaccine. This event lead to a huge number of negative tweets.
Additionally, we observed a notable prevalence of duplicate data within the dataset, with 821 similar tweets recorded on that day.

#### Sinovac

In [ ]:
sinovac = filtered_timeline(df_tweets, 'sinovac', title='Sinovac')

In [ ]:
sinovac_spike = sinovac[sinovac['date'].astype(str)=='2021-06-02']

print('Number of tweets on June 2nd:', len(sinovac_spike))

date_printer(sinovac_spike, ['2021-06-02'], num=10)

Similar to the Covaxin vaccine, the Sinovac vaccine was also approved for emergency use by WHO on June 6th, 2021. This event led to a mostly positive sentiment in the tweets.

#### Moderna

In [ ]:
moderna = filtered_timeline(df_tweets, 'moderna', title='Moderna')

There were numerous spikes related to the Moderna vaccine. Let's focus on the positive spike that happened on June 29th.

In [ ]:
date_printer(moderna, ['2021-06-29'], num=10)

On June 29th, 2021, India granted approval for the importation and utilization of the Moderna vaccine. This development sparked a notable increase in positive tweets.

#### AztraZeneca

In [ ]:
astrazeneca = filtered_timeline(df_tweets, 'astrazeneca', title='AstraZeneca')

#### Sputnik V

In [ ]:
sputnik = filtered_timeline(df_tweets, 'sputnik', title='Sputnik V')


dates = ['2021-04-12', '2021-05-14']

date_printer(sputnik, dates, num=5)

#### Pfizer

In [ ]:
pfizer = filtered_timeline(df_tweets, 'pfizer', title='Pfizer')

In [ ]:
## WORK IN PROGRESS
locations = {
    'India': {'lat': 23.309469, 'long': 78.532748},
    'United States': {'lat': 36.434542, 'long': -103.931671},
    'China': {'lat': 35.377854, 'long': 103.165949},
}
country_counts = {country: 0 for country in locations.keys()}
for index, tweet in df_tweets.iterrows():
    user_location = str(tweet['user_location'])  # Convert to string
    # If the user location contains a country name
    for country in locations.keys():
        if country in user_location:
            country_counts[country] += 1  # Increment count for the country
            break

data = []
for country, count in country_counts.items():
    data.append({'Location': country, 'Latitude': locations[country]['lat'], 'Longitude': locations[country]['long'], 'Number of Tweets': count})

df_locations = pd.DataFrame(data)

print(df_locations)
fig = px.scatter_mapbox(df_locations, lat="Latitude", lon="Longitude", hover_name="Number of Tweets", size = 'Number of Tweets',color="Number of Tweets", 
                    color_continuous_scale=px.colors.sequential.Plasma, size_max=15, zoom=1,
                   mapbox_style="carto-positron")
fig.show()